<a href="https://colab.research.google.com/github/baizhankyzy/academic-knowledge-base/blob/main/knowledge_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Step 1: Replace with your GitHub repo URL
GITHUB_REPO = "https://github.com/baizhankyzy/academic-knowledge-base.git"

#  Step 2: Remove old version if already cloned
!rm -rf academic-knowledge-base

#  Step 3: Clone your GitHub repo
!git clone {GITHUB_REPO}

#  Step 4: Set correct paths for file access
pdf_path = "academic-knowledge-base/data/ai-and-hmi/anticipatory-design.pdf"
output_path = "academic-knowledge-base/output/anticipatory-design.json"


Cloning into 'academic-knowledge-base'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 2.32 MiB | 20.12 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
# 📘 Knowledge Extraction from Academic PDFs to JSON (Google Colab-ready)

# ✅ Section A: Install & Import Packages
!pip install pdfplumber spacy yake -q
import os
import json
import pdfplumber
import spacy
import yake



# ✅ Section C: Load NLP Models
nlp = spacy.load("en_core_web_sm")
kw_extractor = yake.KeywordExtractor()

# ✅ Section D: Extract and Process Metadata from One PDF
def extract_metadata(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

    doc = nlp(text)
    title = text.split("\n")[0][:200]  # First line as fallback title
    keywords = [kw[0] for kw in kw_extractor.extract_keywords(text)[:10]]
    entities = list(set([ent.text for ent in doc.ents]))

    return {
        "title": title,
        "keywords": keywords,
        "entities": entities,
        "full_text": text,
        "file_name": os.path.basename(pdf_path)
    }

# ✅ Section E: Run on Single File and Save as JSON
data = extract_metadata(pdf_path)

with open(output_path, "w") as f:
    json.dump(data, f, indent=4)

print(f"✅ Extracted metadata from {pdf_path} and saved to {output_path}")


✅ Extracted metadata from academic-knowledge-base/data/ai-and-hmi/anticipatory-design.pdf and saved to academic-knowledge-base/output/anticipatory-design.json


## Upload single PDF

In [ ]:
import os
print(os.path.exists("data/ai-and-hmi/anticipatory-design.pdf"))  # Should return True


False


## Extract Raw text with pdf plumber

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])


FileNotFoundError: [Errno 2] No such file or directory: 'data/ai-and-hmi/anticipatory-design.pdf'